In [1]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset = pd.read_csv("DataSetNormalized.csv")

In [3]:
dataset.head()

,user_id,problem_id,level_type,attempts_range,submission_count,problem_solved,contribution,country,follower_count,max_rating,rating,rank
0,user_1,prob_918,E,3,84,73,10,Bangladesh,120,502.007,499.713,advanced
1,user_1,prob_2990,F,3,84,73,10,Bangladesh,120,502.007,499.713,advanced
2,user_1,prob_1358,D,1,84,73,10,Bangladesh,120,502.007,499.713,advanced
3,user_1,prob_4278,A,2,84,73,10,Bangladesh,120,502.007,499.713,advanced
4,user_1,prob_1868,A,1,84,73,10,Bangladesh,120,502.007,499.713,advanced


In [45]:
dataset = dataset.drop(['user_id','problem_id','country'],axis=1)


In [46]:
enc = LabelEncoder()
enc.fit(dataset['level_type'])
dataset['level_type'] = enc.transform(dataset['level_type'])

In [47]:
enc = LabelEncoder()
enc.fit(dataset['rank'])
dataset['rank'] = enc.transform(dataset['rank'])

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-win64) (build 25.121-b15, mixed mode)
  Starting server from C:\Users\sutha\Anaconda3\h2o_jar\h2o.jar
  Ice root: C:\Users\sutha\AppData\Local\Temp\tmp6jtygrdf
  JVM stdout: C:\Users\sutha\AppData\Local\Temp\tmp6jtygrdf\h2o_Dhaval_started_from_python.out
  JVM stderr: C:\Users\sutha\AppData\Local\Temp\tmp6jtygrdf\h2o_Dhaval_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,-05:00
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"1 year, 1 month and 17 days !!!"
H2O cluster name:,H2O_from_python_Dhaval_kmyfo3
H2O cluster total nodes:,1
H2O cluster free memory:,3.530 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [26]:
df = h2o.H2OFrame(dataset)

C:\Users\sutha\Anaconda3\lib\site-packages\h2o\utils\shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
df['attempts_range'] = df['attempts_range'].asfactor()

In [10]:
splits = df.split_frame(ratios=[0.7,0.15],seed=1)

In [11]:
train = splits[0]
valid = splits[1]
test = splits[2]

In [12]:
x = train.columns
y = 'attempts_range'

x.remove(y)

In [13]:
aml1 = H2OAutoML(max_runtime_secs=300, exclude_algos=['DeepLearning'])

In [14]:
aml1.train(x=x,y=y,training_frame=train,leaderboard_frame=test,validation_frame=valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
aml1.leaderboard

model_id,mean_per_class_error
StackedEnsemble_AllModels_0_AutoML_20190426_055119,0.500777
StackedEnsemble_BestOfFamily_0_AutoML_20190426_055119,0.500777
XRT_0_AutoML_20190426_055119,0.504776
DRF_0_AutoML_20190426_055119,0.508662
GLM_grid_0_AutoML_20190426_055119_model_0,0.583283


In [44]:
dataset = pd.read_csv("dataset_final.csv")

In [48]:
df=dataset

In [49]:
df_attempt = df["attempts_range"]
df = df.drop("attempts_range",axis=1)
df.insert(0,"attempts_range",df_attempt)

In [50]:
X = df.iloc[:,1:]
y = df.iloc[:,:1]

In [51]:
test = X[101:102]

In [52]:
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [53]:
y[101:102]

,attempts_range
101,2


In [54]:
aml1.predict(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p1,p2,p3,p4,p5,p6
3,0.000129075,0.000153845,0.500693,0.498836,9.82969e-05,8.99747e-05
